In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [13]:
tf.__version__

'2.0.0'

In [98]:
class NFM(keras.Model):
    def __init__(self,feature_size,field_size,embedding_size):
        super(NFM,self).__init__()
        #self.deep_layers = [32,32]
        self.feature_size = feature_size
        self.embedding_size = embedding_size
        self.field_size = field_size
        self.bias = tf.Variable(tf.constant([0.1]),name='bias')
        #self.bias = tf.constant([0.1])
        ## 
        self.feature_embeddings = tf.Variable(
            tf.random.normal([self.feature_size,self.embedding_size],0.0,0.01),
            name='feature_embeddings')
        self.weight_first = tf.Variable(tf.random.normal([self.feature_size,1],0.0,1.0),name='feature_bias')
        
        #inputs_index = keras.Input((None,))
        #inputs_value = kears.Input((None,))
        
        ##
        self.dense_1 = layers.Dense(32,activation='relu')
        self.dense_2 = layers.Dense(32,activation='relu')
        self.dense_out = layers.Dense(1)
        
    def call(self,inputs_index,inputs_value):
        ## calc embedding
        self.embedding = tf.nn.embedding_lookup(self.feature_embeddings,inputs_index)
        inputs_value = tf.reshape(inputs_value,[self.field_size,-1])
        self.embedding = tf.multiply(self.embedding,inputs_value)
        
        ## first order term
        first_order = tf.nn.embedding_lookup(self.weight_first,inputs_index)
        #inputs_value = tf.reshape(inputs_value,[,-1])
        first_order = tf.reduce_sum(tf.multiply(first_order,inputs_value))
        
        ## second order term 
        sum_feature_emb = tf.reduce_sum(self.embedding,axis=0)
        sum_feature_emb_squre = tf.square(sum_feature_emb)
        
        square_feature_emb = tf.square(self.embedding)
        square_feature_emb_sum = tf.reduce_sum(square_feature_emb,axis=0)
        
        second_order = 0.5 * tf.subtract(sum_feature_emb_squre,square_feature_emb_sum)
        print('second_order',second_order)
        print(second_order.ndim)
        #second_order = layers.Flatten(second_order)
        #print('second_order',second_order)
        second_order = tf.reshape(second_order,[-1,self.embedding_size])
        print('second_order',second_order)
        out_dense = self.dense_1(second_order)
        out_dense = self.dense_2(out_dense)
        out_dense = self.dense_out(out_dense)
        
        
        print('self.bias',self.bias)
        print('first',first_order)
        print('out_dense',out_dense)
        output = tf.add(self.bias,first_order,out_dense) 
        print(output)
        return output
    

In [99]:
import numpy as np
inputs_index = np.array([0,1,2,3],dtype=np.int32)
inputs_value = np.array([0.1,0.3,1.0,1.0],dtype=np.float32)

In [100]:
feature_size = 10
field_size = 4 
embedding_size = 3
model = NFM(feature_size,field_size,embedding_size)

In [101]:
res = model(inputs_index,inputs_value)

second_order tf.Tensor([ 6.1331448e-06 -2.5325287e-05  6.0214297e-05], shape=(3,), dtype=float32)
1
second_order tf.Tensor([[ 6.1331448e-06 -2.5325287e-05  6.0214297e-05]], shape=(1, 3), dtype=float32)
self.bias <tf.Variable 'bias:0' shape=(1,) dtype=float32, numpy=array([0.1], dtype=float32)>
first tf.Tensor(1.2110045, shape=(), dtype=float32)
out_dense tf.Tensor([[1.3354213e-06]], shape=(1, 1), dtype=float32)
tf.Tensor([1.3110045], shape=(1,), dtype=float32)


In [50]:
inputs = keras.Input(shape=(32,))

In [52]:
inputs.shape

TensorShape([None, 32])